# Product Data Analysis

This notebook analyzes the scraped product data from MiSuperFresh.


In [1]:
import json
import pandas as pd
from pathlib import Path
from datetime import datetime

# Set up paths
data_dir = Path("data")
print(f"Data directory: {data_dir.absolute()}")


Data directory: c:\Users\Usuario\Documents\GitHub\web-scrapping super\data


In [2]:
# Find the latest JSON file in the data folder
json_files = list(data_dir.glob("products_*.json"))
if not json_files:
    raise FileNotFoundError("No JSON files found in data folder")

# Sort by modification time and get the latest
latest_json = max(json_files, key=lambda p: p.stat().st_mtime)
print(f"Latest JSON file: {latest_json.name}")
print(f"Modified: {datetime.fromtimestamp(latest_json.stat().st_mtime)}")


Latest JSON file: products_20251118_232608.json
Modified: 2025-11-18 23:26:09.011672


In [3]:
# Load the JSON file
with open(latest_json, "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract metadata and products
metadata = data.get("metadata", {})
products = data.get("products", [])

print(f"Total products: {len(products)}")
print(f"Scraped at: {metadata.get('scraped_at', 'N/A')}")
print(f"Source: {metadata.get('source', 'N/A')}")


Total products: 1922
Scraped at: 2025-11-18T23:26:08.929677
Source: misuperfresh.com.gt


In [4]:
# Create DataFrame
df = pd.DataFrame(products)

# Remove raw_data column if it exists
if "raw_data" in df.columns:
    df = df.drop(columns=["raw_data"])

# Rename columns to Spanish
column_mapping = {
    "barcode": "codigo_barras",
    "category": "categoria",
    "description": "descripcion",
    "image_url": "url_imagen",
    "name": "nombre",
    "offer_description": "descripcion_oferta",
    "offer_price": "precio_oferta",
    "price": "precio",
    "stock": "inventario",
    "subcategory": "subcategoria"
}

df_spanish = df.rename(columns={k: v for k, v in column_mapping.items() if k in df.columns})
df_spanish.head()


,nombre,precio,descripcion,codigo_barras,inventario,precio_oferta,descripcion_oferta,url_imagen,subcategoria,categoria
0,Salsa Ketchup Heinz Botella 20oz,24.45,Salsa Ketchup Heinz Botella 20oz,102749,15.0,21.25,Oferta,https://res.cloudinary.com/gtagt/image/upload/...,Tomate Procesado,Abarrotes
1,Aceite Better Body Foods Almendra Liquida Fras...,102.95,Aceite Better Body Foods Almendra Liquida Fras...,101581,15.0,93.45,Oferta,https://res.cloudinary.com/gtagt/image/upload/...,Aceite Comestible,Abarrotes
2,Aceite Capullo Envase Poly 1500ml,35.85,Aceite Capullo Envase Poly 1500ml,118141,7.0,32.25,Oferta,https://res.cloudinary.com/gtagt/image/upload/...,Aceite Comestible,Abarrotes
3,Aceite Capullo Natural Doy Pack 750ml,21.95,Aceite Capullo Natural Doy Pack 750ml,118143,8.0,16.75,Oferta,https://res.cloudinary.com/gtagt/image/upload/...,Aceite Comestible,Abarrotes
4,Aceite Capullo 100% Vegetal Botella 750ml,21.75,Aceite Capullo 100% Vegetal Botella 750ml,118128,30.0,16.95,Oferta,https://res.cloudinary.com/gtagt/image/upload/...,Aceite Comestible,Abarrotes


In [5]:
# Display summary statistics
print("Estadísticas Resumen:")
print("=" * 60)
print(f"Total productos: {len(df_spanish)}")
print(f"Categorías únicas: {df_spanish['categoria'].nunique() if 'categoria' in df_spanish.columns else 'N/A'}")
print(f"Subcategorías únicas: {df_spanish['subcategoria'].nunique() if 'subcategoria' in df_spanish.columns else 'N/A'}")
print(f"\nEstadísticas de precios:")
if 'precio' in df_spanish.columns:
    prices = pd.to_numeric(df_spanish['precio'], errors='coerce')
    print(f"  Precio mínimo: {prices.min():.2f}")
    print(f"  Precio máximo: {prices.max():.2f}")
    print(f"  Precio promedio: {prices.mean():.2f}")
    print(f"  Precio mediano: {prices.median():.2f}")

if 'precio_oferta' in df_spanish.columns:
    offer_prices = pd.to_numeric(df_spanish['precio_oferta'], errors='coerce')
    products_with_offers = offer_prices.notna().sum()
    print(f"\nProductos con ofertas: {products_with_offers} ({products_with_offers/len(df_spanish)*100:.1f}%)")


Estadísticas Resumen:
Total productos: 1922
Categorías únicas: 13
Subcategorías únicas: 109

Estadísticas de precios:
  Precio mínimo: 0.75
  Precio máximo: 494.95
  Precio promedio: 40.82
  Precio mediano: 24.95

Productos con ofertas: 1922 (100.0%)


In [6]:
# Filter edible products
categorias_no_comestibles = [
    "Bebe",
    "Cuidado Del Hogar / Hogar Y Librería",
    "Cuidado Del Hogar / Limpieza, Ropa Y Hogar",
    "Cuidado Personal",
    "Mascotas",
    "Medicinales"
]

if 'categoria' in df_spanish.columns:
    df_comestibles = df_spanish[~df_spanish['categoria'].isin(categorias_no_comestibles)].copy()
    df_no_comestibles = df_spanish[df_spanish['categoria'].isin(categorias_no_comestibles)].copy()
    
    print("=" * 60)
    print("BREAKDOWN POR TIPO DE PRODUCTO")
    print("=" * 60)
    print(f"Total productos: {len(df_spanish)}")
    print(f"Productos comestibles: {len(df_comestibles)} ({len(df_comestibles)/len(df_spanish)*100:.1f}%)")
    print(f"Productos no comestibles: {len(df_no_comestibles)} ({len(df_no_comestibles)/len(df_spanish)*100:.1f}%)")
else:
    df_comestibles = df_spanish.copy()
    print("Warning: 'categoria' column not found.")


BREAKDOWN POR TIPO DE PRODUCTO
Total productos: 1922
Productos comestibles: 1192 (62.0%)
Productos no comestibles: 730 (38.0%)


In [7]:
# Estadísticas por categoría
if 'categoria' in df_spanish.columns:
    print("=" * 60)
    print("ESTADÍSTICAS POR CATEGORÍA")
    print("=" * 60)
    
    categoria_stats = df_spanish.groupby('categoria').agg({
        'nombre': 'count',
        'precio': lambda x: pd.to_numeric(x, errors='coerce').mean(),
        'precio_oferta': lambda x: pd.to_numeric(x, errors='coerce').mean()
    }).round(2)
    
    categoria_stats.columns = ['Cantidad', 'Precio Promedio', 'Precio Oferta Promedio']
    categoria_stats = categoria_stats.sort_values('Cantidad', ascending=False)
    
    print(categoria_stats.to_string())


ESTADÍSTICAS POR CATEGORÍA
                                            Cantidad  Precio Promedio  Precio Oferta Promedio
categoria                                                                                    
Abarrotes                                        453            27.22                   22.93
Cuidado Personal                                 304            52.85                   42.34
Bebidas                                          273            43.18                   37.00
Lacteos Y Congelados                             203            28.91                   22.51
Cuidado Del Hogar / Limpieza, Ropa Y Hogar       177            35.31                   29.37
Snacks Y Dulces                                  175            21.48                   17.02
Cuidado Del Hogar / Hogar Y Librería             131            52.66                   44.99
Carnes Y Embutidos                                61            28.46                   24.60
Mascotas                         

In [8]:
# Estadísticas por subcategoría (Top 20)
if 'subcategoria' in df_spanish.columns:
    print("=" * 60)
    print("TOP 20 SUBCATEGORÍAS POR CANTIDAD DE PRODUCTOS")
    print("=" * 60)
    
    subcat_stats = df_spanish.groupby('subcategoria').agg({
        'nombre': 'count',
        'precio': lambda x: pd.to_numeric(x, errors='coerce').mean()
    }).round(2)
    
    subcat_stats.columns = ['Cantidad', 'Precio Promedio']
    subcat_stats = subcat_stats.sort_values('Cantidad', ascending=False).head(20)
    
    print(subcat_stats.to_string())


TOP 20 SUBCATEGORÍAS POR CANTIDAD DE PRODUCTOS
                                           Cantidad  Precio Promedio
subcategoria                                                        
Hogar                                           113            40.68
Vinos                                            91           103.88
Boquitas                                         78            17.05
Yogurt                                           76            17.50
Queso, Crema Y Mantequilla                       69            37.81
Shampoos, Acondicionadores Y Tratamientos        65            54.20
Gaseosas                                         61            11.36
Café, Te Y Cremoras                              55            45.57
Detergentes En Polvo Y Liquidos                  48            50.57
Desodorantes                                     47            34.98
Especias Y Sazonadores                           44            10.62
Leche Liquida Uht                                43     

In [9]:
# Análisis de precios detallado
if 'precio' in df_spanish.columns:
    prices = pd.to_numeric(df_spanish['precio'], errors='coerce')
    
    print("=" * 60)
    print("ANÁLISIS DETALLADO DE PRECIOS")
    print("=" * 60)
    print(f"Precio mínimo: Q{prices.min():.2f}")
    print(f"Precio máximo: Q{prices.max():.2f}")
    print(f"Precio promedio: Q{prices.mean():.2f}")
    print(f"Precio mediano: Q{prices.median():.2f}")
    print(f"Desviación estándar: Q{prices.std():.2f}")
    
    print(f"\nPercentiles:")
    print(f"  25%: Q{prices.quantile(0.25):.2f}")
    print(f"  50%: Q{prices.quantile(0.50):.2f}")
    print(f"  75%: Q{prices.quantile(0.75):.2f}")
    print(f"  90%: Q{prices.quantile(0.90):.2f}")
    print(f"  95%: Q{prices.quantile(0.95):.2f}")
    
    print(f"\nDistribución por rangos:")
    print(f"  Q0-10: {(prices <= 10).sum()} productos ({(prices <= 10).sum()/len(prices)*100:.1f}%)")
    print(f"  Q10-25: {((prices > 10) & (prices <= 25)).sum()} productos ({((prices > 10) & (prices <= 25)).sum()/len(prices)*100:.1f}%)")
    print(f"  Q25-50: {((prices > 25) & (prices <= 50)).sum()} productos ({((prices > 25) & (prices <= 50)).sum()/len(prices)*100:.1f}%)")
    print(f"  Q50-100: {((prices > 50) & (prices <= 100)).sum()} productos ({((prices > 50) & (prices <= 100)).sum()/len(prices)*100:.1f}%)")
    print(f"  Q100+: {(prices > 100).sum()} productos ({(prices > 100).sum()/len(prices)*100:.1f}%)")


ANÁLISIS DETALLADO DE PRECIOS
Precio mínimo: Q0.75
Precio máximo: Q494.95
Precio promedio: Q40.82
Precio mediano: Q24.95
Desviación estándar: Q52.58

Percentiles:
  25%: Q12.75
  50%: Q24.95
  75%: Q44.95
  90%: Q91.72
  95%: Q134.90

Distribución por rangos:
  Q0-10: 374 productos (19.5%)
  Q10-25: 603 productos (31.4%)
  Q25-50: 523 productos (27.2%)
  Q50-100: 256 productos (13.3%)
  Q100+: 166 productos (8.6%)


In [10]:
# Análisis de ofertas
if 'precio_oferta' in df_spanish.columns and 'precio' in df_spanish.columns:
    offer_prices = pd.to_numeric(df_spanish['precio_oferta'], errors='coerce')
    regular_prices = pd.to_numeric(df_spanish['precio'], errors='coerce')
    
    discounts = ((regular_prices - offer_prices) / regular_prices * 100).round(2)
    valid_discounts = discounts[discounts > 0]
    
    print("=" * 60)
    print("ANÁLISIS DE OFERTAS")
    print("=" * 60)
    print(f"Productos con oferta: {offer_prices.notna().sum()} ({offer_prices.notna().sum()/len(df_spanish)*100:.1f}%)")
    
    if len(valid_discounts) > 0:
        print(f"\nDescuentos:")
        print(f"  Descuento promedio: {valid_discounts.mean():.1f}%")
        print(f"  Descuento mínimo: {valid_discounts.min():.1f}%")
        print(f"  Descuento máximo: {valid_discounts.max():.1f}%")
        print(f"  Descuento mediano: {valid_discounts.median():.1f}%")
        
        total_savings = (regular_prices - offer_prices).sum()
        print(f"\nAhorro total estimado: Q{total_savings:.2f}")


ANÁLISIS DE OFERTAS
Productos con oferta: 1922 (100.0%)

Descuentos:
  Descuento promedio: 18.7%
  Descuento mínimo: 0.5%
  Descuento máximo: 100.0%
  Descuento mediano: 15.0%

Ahorro total estimado: Q13358.65


In [11]:
# Categorías y subcategorías comestibles detalladas
if 'categoria' in df_comestibles.columns and 'subcategoria' in df_comestibles.columns:
    print("=" * 60)
    print("CATEGORÍAS Y SUBCATEGORÍAS COMESTIBLES")
    print("=" * 60)
    
    categorias_subcategorias = df_comestibles.groupby('categoria')['subcategoria'].unique()
    
    for categoria in sorted(categorias_subcategorias.index):
        subcategorias = categorias_subcategorias[categoria]
        total_in_cat = len(df_comestibles[df_comestibles['categoria'] == categoria])
        
        print(f"\n📁 {categoria} ({len(subcategorias)} subcategorías, {total_in_cat} productos)")
        print("-" * 60)
        
        for subcat in sorted(subcategorias):
            count = len(df_comestibles[(df_comestibles['categoria'] == categoria) & 
                                      (df_comestibles['subcategoria'] == subcat)])
            print(f"  └─ {subcat} ({count} productos)")
    
    print("\n" + "=" * 60)
    print(f"Total categorías comestibles: {df_comestibles['categoria'].nunique()}")
    print(f"Total subcategorías comestibles: {df_comestibles['subcategoria'].nunique()}")
    print(f"Total productos comestibles: {len(df_comestibles)}")


CATEGORÍAS Y SUBCATEGORÍAS COMESTIBLES

📁 Abarrotes (29 subcategorías, 453 productos)
------------------------------------------------------------
  └─ Aceite Comestible (18 productos)
  └─ Aderezos Y Mayonesas (21 productos)
  └─ Atun Y Sardinas Envasadas (20 productos)
  └─ Barras De Cereales (2 productos)
  └─ Café, Te Y Cremoras (55 productos)
  └─ Cereales (41 productos)
  └─ Cereales Calientes / Atoles (14 productos)
  └─ Complementos De Reposteria (13 productos)
  └─ Especias Y Sazonadores (44 productos)
  └─ Frijol Envasado (12 productos)
  └─ Fruta Seca Y Deshidratada (1 productos)
  └─ Frutas Envasadas Y En Almibar (3 productos)
  └─ Gelatinas, Flanes Y Pudines (14 productos)
  └─ Granos Basicos (5 productos)
  └─ Harinas De Trigo (12 productos)
  └─ Leche En Polvo (8 productos)
  └─ Leche Liquida Uht (43 productos)
  └─ Margarinas Y Mantecas (8 productos)
  └─ Miel, Jaleas Y Mermeladas (15 productos)
  └─ Modificadores De Leche (1 productos)
  └─ Pastas Alimenticias (22 prod

In [12]:
# Productos más caros y más baratos
if 'precio' in df_spanish.columns and 'nombre' in df_spanish.columns:
    prices = pd.to_numeric(df_spanish['precio'], errors='coerce')
    df_with_prices = df_spanish.copy()
    df_with_prices['precio_numeric'] = prices
    
    print("=" * 60)
    print("TOP 10 PRODUCTOS MÁS CAROS")
    print("=" * 60)
    top_expensive = df_with_prices.nlargest(10, 'precio_numeric')[['nombre', 'categoria', 'precio']]
    for idx, row in top_expensive.iterrows():
        print(f"  {row['nombre']} - Q{row['precio']} ({row['categoria']})")
    
    print("\n" + "=" * 60)
    print("TOP 10 PRODUCTOS MÁS BARATOS")
    print("=" * 60)
    top_cheap = df_with_prices.nsmallest(10, 'precio_numeric')[['nombre', 'categoria', 'precio']]
    for idx, row in top_cheap.iterrows():
        print(f"  {row['nombre']} - Q{row['precio']} ({row['categoria']})")


TOP 10 PRODUCTOS MÁS CAROS
  Alimento P/Gato Cat Chow Delimix 33lb - Q494.95 (Mascotas)
  Alimento Para Perro Dog Chow Cachorro Med/Gdr Bolsa 20kg - Q484.95 (Mascotas)
  Alimento Para Perro Dog Chow Cachorro Minis Bolsa 20kg - Q484.95 (Mascotas)
  Alimento Para Perro Dog Chow Razas Pequena Bolsa 25kg - Q484.95 (Mascotas)
  Alimento Para Perro Dog Chow Adulto Raza M/G Bolsa 25kg - Q484 (Mascotas)
  Procesador Black&Decker 8 Tazas Color Blanco Fp1337 Caja 1und - Q479.95 (Cuidado Del Hogar / Hogar Y Librería)
  Alimento Para Perro Chispita Alpa Puppy Bolsa 44lb - Q444.95 (Mascotas)
  Cama P/Perro Maskotas 107x10cm Und - Q354.95 (Mascotas)
  Purina Dog Chow Adulto R.G.15kg - Q351.95 (Mascotas)
  Alimento Para Perro Purina Alpo Adulto Carne Asada Bolsa 20kg - Q349.95 (Mascotas)

TOP 10 PRODUCTOS MÁS BARATOS
  Refrescos En Polvo Trix Coctel Sobre 9g - Q0.75 (Bebidas)
  Refrescos En Polvo Trix Fresa Sobre 9g - Q0.75 (Bebidas)
  Refrescos En Polvo Trix Melocoton Sobre 9g - Q0.75 (Bebidas)
  Re

In [13]:
# Imprimir todas las categorías y subcategorías (completo)
print("=" * 60)
print("CATEGORÍAS Y SUBCATEGORÍAS (TODAS)")
print("=" * 60)

if 'categoria' in df_spanish.columns and 'subcategoria' in df_spanish.columns:
    categorias_subcategorias = df_spanish.groupby('categoria')['subcategoria'].unique()
    
    for categoria in sorted(categorias_subcategorias.index):
        subcategorias = categorias_subcategorias[categoria]
        print(f"\nCATEGORIA: {categoria} ({len(subcategorias)} subcategorías)")
        print("-" * 60)
        
        for subcat in sorted(subcategorias):
            count = len(df_spanish[(df_spanish['categoria'] == categoria) & 
                                   (df_spanish['subcategoria'] == subcat)])
            print(f"  SUBCATEGORIA: {subcat} ({count} productos)")


CATEGORÍAS Y SUBCATEGORÍAS (TODAS)

CATEGORIA: Abarrotes (29 subcategorías)
------------------------------------------------------------
  SUBCATEGORIA: Aceite Comestible (18 productos)
  SUBCATEGORIA: Aderezos Y Mayonesas (21 productos)
  SUBCATEGORIA: Atun Y Sardinas Envasadas (20 productos)
  SUBCATEGORIA: Barras De Cereales (2 productos)
  SUBCATEGORIA: Café, Te Y Cremoras (55 productos)
  SUBCATEGORIA: Cereales (41 productos)
  SUBCATEGORIA: Cereales Calientes / Atoles (14 productos)
  SUBCATEGORIA: Complementos De Reposteria (13 productos)
  SUBCATEGORIA: Especias Y Sazonadores (44 productos)
  SUBCATEGORIA: Frijol Envasado (12 productos)
  SUBCATEGORIA: Fruta Seca Y Deshidratada (1 productos)
  SUBCATEGORIA: Frutas Envasadas Y En Almibar (3 productos)
  SUBCATEGORIA: Gelatinas, Flanes Y Pudines (14 productos)
  SUBCATEGORIA: Granos Basicos (5 productos)
  SUBCATEGORIA: Harinas De Trigo (12 productos)
  SUBCATEGORIA: Leche En Polvo (8 productos)
  SUBCATEGORIA: Leche Liquida Uht 